In [1]:
import tensorflow as tf
import numpy as np
import input_data

In [2]:
keep_prob = tf.placeholder('float')
learning_rate = 0.001 # 0.5 진동한다.
training_epochs = 15
batch_size = 100
display_step = 1

In [3]:
mnist = input_data.read_data_sets("MNIST_DATA/", one_hot=True)

Extracting MNIST_DATA/train-images-idx3-ubyte.gz
Extracting MNIST_DATA/train-labels-idx1-ubyte.gz
Extracting MNIST_DATA/t10k-images-idx3-ubyte.gz


/usr/local/Cellar/python3/3.6.0/Frameworks/Python.framework/Versions/3.6/lib/python3.6/gzip.py:276: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  return self._buffer.read(size)
/Users/yjhong/Documents/work/2016/dl_everyone/lab10/input_data.py:35: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)


Extracting MNIST_DATA/t10k-labels-idx1-ubyte.gz


## initialize vars - xavier initialize

In [4]:
def xavier_init(n_inputs, n_outputs, uniform=True):
    if uniform:
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev = stddev)

X = tf.placeholder('float', [None, 784])
Y = tf.placeholder('float', [None, 10])
training = tf.placeholder('bool')

W1 = tf.get_variable("W1", shape=[784, 256], initializer = xavier_init(784, 256))
b1 = tf.Variable(tf.random_normal([256]))
W2 = tf.get_variable("W2", shape=[256, 256], initializer = xavier_init(256, 256))
b2 = tf.Variable(tf.random_normal([256]))
W3 = tf.get_variable("W3", shape=[256, 256], initializer = xavier_init(256, 256))
b3 = tf.Variable(tf.random_normal([256]))
W4 = tf.get_variable("W4", shape=[256, 256], initializer = xavier_init(256, 256))
b4 = tf.Variable(tf.random_normal([256]))
W5 = tf.get_variable("W5", shape=[256, 10], initializer = xavier_init(256, 10))
b5 = tf.Variable(tf.random_normal([10]))


### dropout

In [5]:

L1 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(X, W1), b1)), keep_prob)
L2 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(L1, W2), b2)), keep_prob)
L3 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(L2, W3), b3)), keep_prob)
L4 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(L3, W4), b4)), keep_prob)
'''
# 모든 레이어마다 dropout을 해 줘야 하나? 마지막 layer만 해 주면 안되나?
L1 = tf.nn.relu(tf.add(tf.matmul(X, W1), b1))
L2 = tf.nn.relu(tf.add(tf.matmul(L1, W2), b2))
L3 = tf.nn.relu(tf.add(tf.matmul(L2, W3), b3))
L4 = tf.nn.relu(tf.add(tf.matmul(L3, W4), b4))
'''

y_ = tf.add(tf.matmul(L4, W5), b5)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_, Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

### Batch Normalization

In [ ]:
L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.contrib.layers.batch_norm(L1, is_training = training, center=True, scale=True, activation_fn=tf.nn.relu, updates_collections=None, decay=0.9)
L1 = tf.nn.relu(L1)
L2 = tf.add(tf.matmul(L1, W2), b2)
L2 = tf.contrib.layers.batch_norm(L2, is_training = training, center=True, scale=True, activation_fn=tf.nn.relu, updates_collections=None, decay=0.9)
L2 = tf.nn.relu(L2)
L3 = tf.add(tf.matmul(L2, W3), b3)
L3 = tf.contrib.layers.batch_norm(L3, is_training = training, center=True, scale=True, activation_fn=tf.nn.relu, updates_collections=None, decay=0.9)
L3 = tf.nn.relu(L3)
L4 = tf.add(tf.matmul(L3, W4), b4)
L4 = tf.contrib.layers.batch_norm(L4, is_training = training, center=True, scale=True, activation_fn=tf.nn.relu, updates_collections=None, decay=0.9)
L4 = tf.nn.relu(L4)

y_ = tf.add(tf.matmul(L4, W5), b5)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_, Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

In [6]:
correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

In [7]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [8]:
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        cost_val, _ = sess.run([cost, optimizer], feed_dict = {X:batch_xs, Y:batch_ys, keep_prob: 0.5, training:True})
        avg_cost += cost_val / total_batch
    if (epoch % display_step == 0):
        print("Epoch:", '%04d' %(epoch + 1), "cost=", "{:.9f}".format(avg_cost))
print("Optimization Finished")

Epoch: 0001 cost= 1.060816654
Epoch: 0002 cost= 0.399496094
Epoch: 0003 cost= 0.305033097
Epoch: 0004 cost= 0.256682840
Epoch: 0005 cost= 0.229955318
Epoch: 0006 cost= 0.210138147
Epoch: 0007 cost= 0.195983493
Epoch: 0008 cost= 0.183697381
Epoch: 0009 cost= 0.173968045
Epoch: 0010 cost= 0.165770853
Epoch: 0011 cost= 0.158796339
Epoch: 0012 cost= 0.156630205
Epoch: 0013 cost= 0.147133121
Epoch: 0014 cost= 0.141484219
Epoch: 0015 cost= 0.138041487
Optimization Finished


In [9]:
sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels, keep_prob:1., training:False})

0.9781

In [10]:
sess.close()